In [1]:
# https://scores.readthedocs.io/en/stable/tutorials/Fractions_Skill_Score.html

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pathlib import Path
from datetime import datetime
import re
import pandas as pd
from scores.spatial import fss_2d

from dask.distributed import Client, wait
import os, sys
import psutil

sys.path.append('/home/548/cd3022/repos/Irradiance-comparisons/Irradiance-comparisons')
import logger
from read_datasets import read_dataset

LOG = logger.get_logger(__name__)

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.06/lib/python3.11/site-packages/pyproj/__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_DATA (PROJ 9.1+) | PROJ_LIB (PROJ<9.1) or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))
ERROR 1: PROJ: proj_create_from_database: Open of /g/data/xp65/public/./apps/med_conda/envs/analysis3-25.06/share/proj failed
PROJ: proj_create_from_database: Open of /g/data/xp65/public/./apps/med_conda/envs/analysis3-25.06/share/proj failed


# Start Dask Client

In [2]:
client = Client(
    n_workers=24,
    threads_per_worker=1
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 24
Total threads: 24,Total memory: 250.26 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39705,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:42627,Total threads: 1
Dashboard: http://127.0.0.1:35903/status,Memory: 10.43 GiB
Nanny: tcp://127.0.0.1:34469,


# Get Lat/Lon from Ancillary File

In [3]:
# the ancillary that contains the lat/lon coordinates that line up with ny/nx
ancil = xr.open_dataset('/g/data/ra22/satellite-products/arc/obs/himawari-ahi/fldk/latest/ancillary/00000000000000-P1S-ABOM_GEOM_SENSOR-PRJ_GEOS141_2000-HIMAWARI8-AHI.nc')
ancil = ancil.isel(time=0)

2025-11-06 16:52:15,202:py.warnings:WARNING: /g/data/xp65/public/apps/med_conda/envs/analysis3-25.06/lib/python3.11/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)

2025-11-06 16:52:15,202:py.warnings:WARNING: /g/data/xp65/public/apps/med_conda/envs/analysis3-25.06/lib/python3.11/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)



# Assign Lat, Lon, and Time

In [4]:
path = Path('/g/data/rv74/satellite-products/arc/der/himawari-ahi/cloud/cmic/latest/2020/01/')
files = sorted(path.rglob("*.nc"))[0:1]

# Pre-load ancillary coordinates once
lat_ancil = ancil.lat.data
lon_ancil = ancil.lon.data

 
def make_preprocess_func(filepaths):
    def _preprocess(ds, file_idx):
        f = filepaths[file_idx]
        match = re.search(r"_(\d{8}T\d{6})Z\.nc$", f.name)
        time = pd.to_datetime(match.group(1), format="%Y%m%dT%H%M%S")
        ds = ds.assign_coords(
            time=time,
            lat=(("ny", "nx"), lat_ancil),
            lon=(("ny", "nx"), lon_ancil)
        )
        ds["cmic_lwp"] = ds["cmic_lwp"].fillna(0)
        return ds

    def preprocess_with_index(i, ds):
        return _preprocess(ds, i)

    return preprocess_with_index


# Open all files lazily with Dask
preprocess_func = make_preprocess_func(files)

datasets = []
for i, f in enumerate(files):
    ds = xr.open_dataset(f, chunks="auto")
    ds = preprocess_func(i, ds)
    datasets.append(ds)

ds = xr.concat(datasets, dim="time")

# Optional: Regrid to get Lat/Lon into Dimensions

In [ ]:
import xesmf as xe

# Define target 1D grid
target = xr.Dataset({
    "lat": (["lat"], np.linspace(lat_min, lat_max, bar.lat.shape[0])),
    "lon": (["lon"], np.linspace(lon_min, lon_max, bar.lon.shape[0])),
})

regridder = xe.Regridder(ds, target, "bilinear")
ds_reg = regridder(ds)

ds_reg = ds_reg.interp(
    lat=bar.lat,
    lon=bar.lon,
    method='linear'
)